In [ ]:
!apt install ffmpeg
!pip install spleeter
!pip install librosa
!pip install pydub
!pip install SpeechRecognition
from pydub import AudioSegment
import numpy as np
import csv
import librosa

In [ ]:
!spleeter separate /content/vocals.wav-o output/

ERROR:spleeter:An error occurs with ffprobe (see ffprobe output below)

ffprobe version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2007-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
# Load the audio file (replace 'audio_file_path' with the actual path)
audio_file_path = '/vocals.wav'
audio = AudioSegment.from_file(audio_file_path, format="wav")

# Perform speech recognition using the SpeechRecognition library
recognizer = sr.Recognizer()
with sr.AudioFile(audio_file_path) as source:
    audio_data = recognizer.record(source)

text = recognizer.recognize_google(audio_data, language='hi-IN')

# Estimate timestamps based on word count
words = text.split()

num_words=len(words)
print(num_words)

word_duration = len(audio) / len(words)

timestamps = []
for i, word in enumerate(words):
    start_time = i * word_duration
    end_time = (i + 1) * word_duration
    timestamps.append((word, start_time, end_time))

# Print estimated timestamps
print("Estimated Word Timestamps:")
for word, start, end in timestamps:
    print(f"Word: {word} | Start Time: {start} ms | End Time: {end} ms")


15
Estimated Word Timestamps:
Word: तेरे | Start Time: 0.0 ms | End Time: 4176.466666666666 ms
Word: नाल | Start Time: 4176.466666666666 ms | End Time: 8352.933333333332 ms
Word: दिया | Start Time: 8352.933333333332 ms | End Time: 12529.399999999998 ms
Word: मेनू | Start Time: 12529.399999999998 ms | End Time: 16705.866666666665 ms
Word: जान | Start Time: 16705.866666666665 ms | End Time: 20882.333333333332 ms
Word: दिया | Start Time: 20882.333333333332 ms | End Time: 25058.799999999996 ms
Word: तेरे | Start Time: 25058.799999999996 ms | End Time: 29235.266666666663 ms
Word: मेरे | Start Time: 29235.266666666663 ms | End Time: 33411.73333333333 ms
Word: नैनो | Start Time: 33411.73333333333 ms | End Time: 37588.2 ms
Word: पहचान | Start Time: 37588.2 ms | End Time: 41764.666666666664 ms
Word: दिया | Start Time: 41764.666666666664 ms | End Time: 45941.13333333333 ms
Word: तू | Start Time: 45941.13333333333 ms | End Time: 50117.59999999999 ms
Word: सुना | Start Time: 50117.59999999999 ms |

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the audio file (replace 'audio_file_path' with the actual path)
audio_file_path = '/content/drive/MyDrive/Project Sample audio/Aj Jane.wav'
audio = AudioSegment.from_file(audio_file_path, format="wav")

# Perform speech recognition using the SpeechRecognition library
recognizer = sr.Recognizer()
with sr.AudioFile(audio_file_path) as source:
    audio_data = recognizer.record(source)

text = recognizer.recognize_google(audio_data, language='hi-IN')

# Split the recognized text into words
words = text.split()

num_words=len(words)

# Calculate word timestamps as before
word_duration = len(audio) / len(words)
timestamps = []
for i, word in enumerate(words):
    start_time = i * word_duration
    end_time = (i + 1) * word_duration
    timestamps.append((word, start_time, end_time))

# Calculate the fundamental frequency (pitch) of each word
word_pitch = {}
for word, start, end in timestamps:
    word_audio = audio[int(start):int(end)]
    y, sr = librosa.load(word_audio.export(format="wav"), sr=None)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    # Extract the pitch (frequency) information
    pitch = np.mean(pitches)
    word_pitch[word] = pitch

# Convert pitch (frequency) into corresponding musical notes
def frequency_to_note_name(frequency):
    if frequency <= 0:
        return "Unknown"
    n = 12 * (np.log2(frequency / 440)) + 49
    note_names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    octave = int(n // 12)
    note = int(n % 12)
    return f"{note_names[note]}{octave}"

# Print estimated timestamps, exact frequency, and corresponding musical notes
print("Estimated Word Timestamps, Frequencies, and Musical Notes:")
for word, start, end in timestamps:
    pitch = word_pitch[word]
    note = frequency_to_note_name(pitch)
    print(f"Word: {word} | Start Time: {start} ms | End Time: {end} ms | Pitch (Frequency): {pitch:.2f} Hz | Note: {note}")


In [ ]:
#headers contains the names of the columns
'''Pid-unique id to recognise each element
   Words-contains corresponding words
   s_time- starting time
   e_time- ending time
   time- e_time-s_time
   frequency- avg tonal frequency of the words
   freq_diff- frequency diff
'''
headers=["Pid","Words","s_time","e_time","time","frequency","freq_diff"]
with open('names.csv','w')as csv_file:
    song_csv=csv.writer(csv_file,delimiter='-')
    song_csv.writerow(headers)
    for i in range(0,num_words):

      song_csv.writerows()